# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 2**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 2. Application de la RI pour la liaison référentielle

Une application des méthodes de RI est la liaison référentielle. Cette tâche consiste à trouver le lien entre une portion de texte et la page Wikipédia qui le représentent le mieux. Dans ce cas d’application :
* Comme requêtes nous allons utiliser chaque mot dans une phrase (ou chaque mot important).
* Le corpus sont les pages Wikipédia. Donc, chaque page Wikipédia est un document à retrouver.
* La liste de résultats sont les candidats pour la liaison référentielle.

Considère par exemple la chaîne de caractères « ***Thomas and Mario are strikers playing in Munich*** ». Il existe trois sous-chaînes (***Thomas***, ***Mario*** et ***Munich***) ou mots importants qui sont susceptibles d'être identifiées dans une base de connaissances telle que Wikipédia. Une recherche de ces mots-clés dans les pages Wikipédia produirait de nombreux concepts candidats pour ces sous-chaînes. Donc un bon système doit retrouver comme premier résultats les liens aux pages Wikipédia : [Thomas Müller](https://en.wikipedia.org/wiki/Thomas_M%C3%BCller), [Mario Gómez](https://en.wikipedia.org/wiki/Mario_G%C3%B3mez) (ou Mario Götze) et FC [Bayern Munich](https://en.wikipedia.org/wiki/FC_Bayern_Munich). Cette tâche est particulièrement complexe et les méthodes de recherche d’information sont particulièrement utiles pour la résoudre.

> Plusieurs méthodes de liaison référentielle (Entity Linking en anglais) sont disponibles sur internet. Les plus connus sont :

>	* Spotlight → https://demo.dbpedia-spotlight.org/
>	* Babelfy → http://babelfy.org/
>	* TagMe → https://tagme.d4science.org/tagme/

Étapes à suivre pendant ce TP :
1. Liaison avec le texte
2. Les poids
3. Utilisation de termes
4. Le titre
5. L'outil


## 0. Installation de l’environnement de travail

Installation et configuration de librairies comment le TP précédent.

In [30]:
# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [31]:
#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-1b4dl1_2/python-terrier_100a9c8ef0d24bbe9cd7e922864a4c5f
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git /tmp/pip-install-1b4dl1_2/python-terrier_100a9c8ef0d24bbe9cd7e922864a4c5f
  Resolved https://github.com/terrier-org/pyterrier.git to commit 81f20bda483f6044bda89f717ae863e62bab42e5
  Preparing metadata (setup.py) ... done


In [32]:
#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()

### Lecture d'un index existant.

Ajoutez le code pour charger l'index créé sur **text** dans le TP 1.

In [33]:
!ls -lh

total 54M
drwx------ 5 root root 4.0K Mar 20 16:30 drive
drwxr-xr-x 2 root root 4.0K Mar  7 15:41 pd_index
-rw-r--r-- 1 root root  54M Mar 20 15:09 pd_index.zip
drwxr-xr-x 1 root root 4.0K Mar 18 13:23 sample_data


In [34]:
# 1. Ajoutez votre pd_index.zip du TP1 à Google Colab

# 2. Changer NOM REPERTOIRE par le nom de votre dossier. Après exécuté la commande suivante
!unzip -o pd_index.zip

Archive:  pd_index.zip
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.document.fsarrayfile  
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.properties  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.lexicon.fsomaphash  
  inflating: pd_index/data.meta.idx  
  inflating: pd_index/data.inverted.bf  


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
indexref2 = pt.IndexRef.of("/content/pd_index/data.properties") # ICI VOTRE CODE

# 1) Liaison avec le texte
En utilisant l’indexe du champ **text** de Wikipédia (comme dans le TP1) et la requête *« Thomas and Mario are strikers playing in Munich »* vérifiez si vous obtenez les pages Wikipédia respectives (comme le feraient Spotlight, Babelfy ou TagMe) :
* Thomas Müller
* Mario Gómez ou Mario Götze
* FC Bayern Munich

Voici la requete sur l'index mentionné :

In [39]:
pt.BatchRetrieve(indexref2, wmodel="BM25", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,30.771667,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,30.306808,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,29.796582,Thomas and Mario are strikers playing in Munich
3,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,3,28.570535,Thomas and Mario are strikers playing in Munich
4,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,4,28.192863,Thomas and Mario are strikers playing in Munich
5,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,5,27.400834,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,25.804927,Thomas and Mario are strikers playing in Munich
7,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,7,25.073465,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,24.388908,Thomas and Mario are strikers playing in Munich
9,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,9,23.780821,Thomas and Mario are strikers playing in Munich


....

ICI VOTRE RÉPONSE POUR LA COMPARAISON DES RÉSULTATS :
- Ce que je vois c'est que il y a que Thomas Mller dans les resultats les deux autres ne font pas partie

....

# 2) Les poids

Utilisez les différentes méthodes de pondération prédéfinies sur Pyterrier (http://terrier.org/docs/v5.2/javadoc/org/terrier/matching/models/package-summary.html) pour voir s’il existe des méthodes que vous donnent de meilleurs résultats.

Choisir 5 méthodes (à vous de décider lesquels) et les comparer.

C’est-à-dire inspectez les 10 premiers résultats et vérifiez si les pages d’intérêt (Thomas Müller, Mario Gómez ou Mario Götze et FC Bayern Munich) font partie des résultats, puis les 20 premiers, et finalement les 50 premiers. Existent-ils de différences importantes ?

In [40]:
## ICI VOTRE CODE
pt.BatchRetrieve(indexref2, wmodel="LGD", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
# 10 rien
# 20 rien
# 50 Mario Gmez  Thomas Mller  Mario Gtze

,qid,docid,docno,title,url,rank,score,query
0,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,0,26.279767,Thomas and Mario are strikers playing in Munich
1,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,1,24.896580,Thomas and Mario are strikers playing in Munich
2,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,2,23.312584,Thomas and Mario are strikers playing in Munich
3,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,3,23.195746,Thomas and Mario are strikers playing in Munich
4,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,4,23.142737,Thomas and Mario are strikers playing in Munich
5,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,5,22.866804,Thomas and Mario are strikers playing in Munich
6,1,7608,24698,Luigi,https://simple.wikipedia.org/wiki/Luigi,6,22.360247,Thomas and Mario are strikers playing in Munich
7,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,7,22.097048,Thomas and Mario are strikers playing in Munich
8,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,8,22.031574,Thomas and Mario are strikers playing in Munich
9,1,105771,459770,UEFA Euro 2020,https://simple.wikipedia.org/wiki/UEFA%20Euro%...,9,21.851881,Thomas and Mario are strikers playing in Munich


In [41]:
pt.BatchRetrieve(indexref2, wmodel="DLH", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
# 10 rien
# 20 rien
# 50 Thomas Mller

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,16.484302,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,16.188615,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,16.054922,Thomas and Mario are strikers playing in Munich
3,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,3,15.371080,Thomas and Mario are strikers playing in Munich
4,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,4,15.233306,Thomas and Mario are strikers playing in Munich
5,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,5,15.205449,Thomas and Mario are strikers playing in Munich
6,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,6,14.790709,Thomas and Mario are strikers playing in Munich
7,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,7,14.468759,Thomas and Mario are strikers playing in Munich
8,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,8,14.152757,Thomas and Mario are strikers playing in Munich
9,1,7608,24698,Luigi,https://simple.wikipedia.org/wiki/Luigi,9,13.449981,Thomas and Mario are strikers playing in Munich


In [42]:
pt.BatchRetrieve(indexref2, wmodel="Dl", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
# 10 rien
# 20 rien
# 50 rien

,qid,docid,docno,title,url,rank,score,query
0,1,114470,513675,List of people from Texas,https://simple.wikipedia.org/wiki/List%20of%20...,0,87810.0,Thomas and Mario are strikers playing in Munich
1,1,121822,565509,List of accidents and incidents involving comm...,https://simple.wikipedia.org/wiki/List%20of%20...,1,33156.0,Thomas and Mario are strikers playing in Munich
2,1,49319,178089,2020,https://simple.wikipedia.org/wiki/2020,2,31158.0,Thomas and Mario are strikers playing in Munich
3,1,86581,377871,Deaths in 2013,https://simple.wikipedia.org/wiki/Deaths%20in%...,3,28707.0,Thomas and Mario are strikers playing in Munich
4,1,114308,512797,List of people from New Jersey,https://simple.wikipedia.org/wiki/List%20of%20...,4,28449.0,Thomas and Mario are strikers playing in Munich
5,1,51419,188999,2021,https://simple.wikipedia.org/wiki/2021,5,26026.0,Thomas and Mario are strikers playing in Munich
6,1,160398,755400,Deaths in December 2020,https://simple.wikipedia.org/wiki/Deaths%20in%...,6,22473.0,Thomas and Mario are strikers playing in Munich
7,1,177064,804759,Deaths in August 2021,https://simple.wikipedia.org/wiki/Deaths%20in%...,7,20584.0,Thomas and Mario are strikers playing in Munich
8,1,177059,804754,Deaths in March 2021,https://simple.wikipedia.org/wiki/Deaths%20in%...,8,15105.0,Thomas and Mario are strikers playing in Munich
9,1,177057,804752,Deaths in January 2021,https://simple.wikipedia.org/wiki/Deaths%20in%...,9,14828.0,Thomas and Mario are strikers playing in Munich


In [ ]:
pt.BatchRetrieve(indexref2, wmodel="DFIZ", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
# 10 rien
# 20 rien
# 50 Thomas Mller

,qid,docid,docno,title,url,rank,score,query
0,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,0,13.526372,Thomas and Mario are strikers playing in Munich
1,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,1,13.501512,Thomas and Mario are strikers playing in Munich
2,1,1066,4016,Mario,https://simple.wikipedia.org/wiki/Mario,2,13.425485,Thomas and Mario are strikers playing in Munich
3,1,7608,24698,Luigi,https://simple.wikipedia.org/wiki/Luigi,3,13.025328,Thomas and Mario are strikers playing in Munich
4,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,4,12.731638,Thomas and Mario are strikers playing in Munich
5,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,5,12.520257,Thomas and Mario are strikers playing in Munich
6,1,93934,409469,Charles Martinet,https://simple.wikipedia.org/wiki/Charles%20Ma...,6,12.408279,Thomas and Mario are strikers playing in Munich
7,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,7,12.235782,Thomas and Mario are strikers playing in Munich
8,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,8,11.946119,Thomas and Mario are strikers playing in Munich
9,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,9,11.930306,Thomas and Mario are strikers playing in Munich


In [ ]:
pt.BatchRetrieve(indexref2, wmodel="BB2", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
# 10 rien
# 20 rien
# 50 rien

,qid,docid,docno,title,url,rank,score,query
0,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,0,30.077018,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,29.431805,Thomas and Mario are strikers playing in Munich
2,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,2,29.377400,Thomas and Mario are strikers playing in Munich
3,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,3,28.709982,Thomas and Mario are strikers playing in Munich
4,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,4,28.658027,Thomas and Mario are strikers playing in Munich
5,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,5,28.361979,Thomas and Mario are strikers playing in Munich
6,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,6,27.841325,Thomas and Mario are strikers playing in Munich
7,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,7,27.424951,Thomas and Mario are strikers playing in Munich
8,1,7608,24698,Luigi,https://simple.wikipedia.org/wiki/Luigi,8,27.211428,Thomas and Mario are strikers playing in Munich
9,1,1066,4016,Mario,https://simple.wikipedia.org/wiki/Mario,9,26.212239,Thomas and Mario are strikers playing in Munich


....

ICI VOTRE RÉPONSE:
- Methode : LGD
# 10 rien
# 20 rien
# 50 on trouve Mario Gmez et  Thomas Mller et  Mario Gtze  

- Methode : DLH
# 10 rien
# 20 rien
# 50 Thomas Mller

- Methode : Dl
# 10 rien
# 20 rien
# 50 rien

- Methode : DFIZ
# 10 rien
# 20 rien
# 50 Thomas Mller

- Methode : BB2
# 10 rien
# 20 rien
# 50 rien



....

# 3) Utilisation de termes

Une stratégie consiste à utiliser individuellement chaque terme de la requête pour voir si vous arrivez à trouver les pages d’intérêt. Un code exemple est donné ci-dessous, cependant, il n'est pas suffisant pour faire la tâche.

Proposer une modification de cette fonction basique (à vous de vous inspirer/inventer des modifications qui améliorent les résultats)

Une fois fini votre code, réalisez la même vérification de résultats que dans le point précédent.


In [43]:
# MODIFIER CETTE FONCTION
def basic_term_search(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno","title","url"]).search(term)[['title','score']]
    print(dfres.head(10))
    print("Position de la page attendue dans les résultats :\n",dfres[dfres['title']==page],"\n\n\n")


query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")

['Thomas', 'Mario', 'Munich']
                         title      score
0             Thomas (surname)  11.700841
1                 Saint Thomas  11.597729
2           Thomas the Apostle  11.447421
3                Thomas County  11.387148
4           Thomas Wedgwood IV  11.210921
5                 Sarah Thomas  11.152142
6                 Helen Thomas  11.145990
7             Gospel of Thomas  11.121265
8      Thomas White (merchant)  11.026214
9  Mallappally Marthoma Church  11.013052
Position de la page attendue dans les résultats :
 Empty DataFrame
Columns: [title, score]
Index: [] 



                          title      score
0                Princess Peach  16.478378
1        Mario (disambiguation)  16.451796
2  Super Mario (disambiguation)  16.419316
3                         Mario  16.395732
4              Charles Martinet  16.349001
5                    Bowser Jr.  16.214048
6                         Wario  16.177717
7                         Luigi  16.132403
8               

In [44]:
# ICI VOTRE CODE POUR LA COMPARAISON
# Nouvelle Version
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Assurez-vous de télécharger les ressources nécessaires
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_query(query):
    stop_words = set(stopwords.words('english'))
    query_tokens = word_tokenize(query)
    filtered_tokens = [word for word in query_tokens if word.lower() not in stop_words]
    return " ".join(filtered_tokens)

def basic_term_search(query, method):
    preprocessed_query = preprocess_query(query)
    # Recherche par phrase
    dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno", "title", "url"]).search(preprocessed_query)[['title', 'score']]
    # Tri des résultats par score décroissant
    dfres_sorted = dfres.sort_values(by='score', ascending=False)
    print(dfres_sorted.head(10))
    # Vérifiez si les pages attendues sont dans les résultats
    expected_pages = ['Thomas Mller', 'Mario Gtze', 'FC Bayern Munich']
    for page in expected_pages:
        print("Position de la page attendue dans les résultats :")
        print(dfres_sorted[dfres_sorted['title'].str.contains(page)], "\n\n\n")

query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                    title      score
0  Mario Strikers Charged  30.771667
1    Mario Smash Football  30.306808
2           Audi Cup 2013  29.796582
3             Donkey Kong  28.570535
4          Princess Daisy  28.192863
5         Mario Balotelli  27.400834
6                   Yoshi  25.804927
7          Princess Peach  25.073465
8           Audi Cup 2015  24.388908
9       Alexander Zickler  23.780821
Position de la page attendue dans les résultats :
           title      score
35  Thomas Mller  18.725876 



Position de la page attendue dans les résultats :
          title      score
311  Mario Gtze  14.562551 



Position de la page attendue dans les résultats :
                              title      score
255             FC Bayern Munich II  15.109127
463                FC Bayern Munich  13.506697
574  202122 FC Bayern Munich season  12.547807
658  201920 FC Bayern Munich season  11.971975
689        FC Bayern Munich (women)  11.791081
718  202021 FC Bayern Munich season  11.667

....

ICI VOTRE RÉPONSE :
- Ici on peut voit que dans la version ameliore le code a reussi de trouver les 3 : Thmas Mller et Mario Gtze et Fc Bayern Munich

# 4) Le titre
Maintenant utilisez le champ ***title*** avec la requête complète et avec chaque terme. Expliquez  pourquoi les résultats sont différents.

In [48]:
# Installation de la librairie datasets
!pip -q install datasets
#!pip -q install apache_beam==2.47.0

In [49]:
import pandas as pd
from datasets import load_dataset

In [50]:
wksimple = load_dataset("wikipedia", "20220301.simple")
df = pd.DataFrame(wksimple['train'])
df.columns = ['docno','url','title','text']

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [51]:
#Création du dossier pour le stockage des indexes

# Attention ! Pour remplacer un index précedent, il faut l'effacer en utilisant : !rm -rf ./pd_index
# Cliquez sur l'icône des fichiers dans la barre de gauche pour voir tous les fichiers disponibles dans cette session Colab.
# Vous verrez qu'après l'indexation, le fichier 'pd_index' est créé.

#pd_title_indexer = pt.DFIndexer("/content/drive/MyDrive/title_index")

In [52]:
indexref3 = pt.IndexRef.of("/content/drive/MyDrive/title_index/data.properties") # ICI VOTRE CODE

In [53]:
indexref3 = indexref2

In [54]:
# MODIFIER CETTE FONCTION
def basic_term_search(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref3, wmodel=method, metadata=["url"]).search(term)
    print(dfres.head(10))


query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")

['Thomas', 'Mario', 'Munich']
  qid   docid                                                url  rank  \
0   1   91931  https://simple.wikipedia.org/wiki/Thomas%20%28...     0   
1   1   11169   https://simple.wikipedia.org/wiki/Saint%20Thomas     1   
2   1    8167  https://simple.wikipedia.org/wiki/Thomas%20the...     2   
3   1  167355  https://simple.wikipedia.org/wiki/Thomas%20County     3   
4   1   15129  https://simple.wikipedia.org/wiki/Thomas%20Wed...     4   
5   1  199625   https://simple.wikipedia.org/wiki/Sarah%20Thomas     5   
6   1   94201   https://simple.wikipedia.org/wiki/Helen%20Thomas     6   
7   1   27604  https://simple.wikipedia.org/wiki/Gospel%20of%...     7   
8   1   99243  https://simple.wikipedia.org/wiki/Thomas%20Whi...     8   
9   1  162167  https://simple.wikipedia.org/wiki/Mallappally%...     9   

       score   query  
0  11.700841  Thomas  
1  11.597729  Thomas  
2  11.447421  Thomas  
3  11.387148  Thomas  
4  11.210921  Thomas  
5  11.152142  Tho

In [ ]:
#pt.BatchRetrieve(indexref3, wmodel="BM25", metadata=["docno","url","text"]).search("Thomas and Mario are strikers playing in Munich").head(50)

....

ICI VOTRE RÉPONSE

....

# 5) L'outil
Développez un outil/méthode/système capable de recevoir comme entrée un paragraphe de texte et comme sortie une liste ordonné de pages Wikipédia pour chaque unité lexical (token ou mot) dans le paragraphe.



---



En plus à l’information du TP, vous pouvez utiliser les ressources/idées suivants pour améliorer les résultats du système à développer :
- La combinaison de plusieurs méthodes de pondération c’est possible en utilisant les indications sur https://pyterrier.readthedocs.io/en/latest/pipeline_examples.html .
- Comme mentionné dans le TP1, il existe plusieurs type de champs. On a utilisé celui de la classe TextField. Renseignez vous sur les autres (StringField par exemple) et ses différences.
- Les distances d’édition sont largement utilisés par les moteurs de recherche. Par exemple si vous tapez « paris sain germain » sur Google il va vous suggérer la requête « paris saint germain » l’équipe de football. Vous pouvez utiliser les distances d’édition pour trouver des entités avec une orthographe différent que celle sur Wikipédia. Une de distance très performantes est la distance Levenshtein implémentée dans la classe LuceneLevenshteinDistance du paquet lucene.search.spell.
- Les moteurs de recherche peuvent être utilisés pour calculer la similarité entre les mots ou entre les entités. Par exemple, sur Google si vous tapez « orange » et « carrot » il y a plus de résultats que si vous tapez « purple » et « carrot ». Vous pouvez utiliser cette information pour décider si deux pages Wikipédia font partie d’un même sujet.
- Utilisez le fichier [anchor_text_en.ttl](http://downloads.dbpedia.org/2015-10/core-i18n/en/anchor_text_en.ttl.bz2) qui contient les anchor texts  de la Wikipédia en anglais. Le format du fichier est décrit [ici](https://iai-group.github.io/DBpedia-Entity/index_details.html). Transformez-le dans ce format au autre similaire pour faire l'indexation:

|             Frequency                 	|       Wikipedia Page        	| Anchor Text 	|
|------------------------------	|---------------	|-----------	|
|162| \<Saturn_Award_for_Best_Supporting_Actor> |"Best Supporting Actor"|
|14 |\<André_Golke>| "Golke"|
|13 |\<Charles_Pinckney_National_Historic_Site> |"Charles Pinckney National Historic Site"|
|25 |\<Thomas_Manfredini>| "Manfredini"|
|13 |<Marek_Zieńczuk> |"Marek Zieńczuk"|
|14 |\<Sigurd_Ulveseth>| "Sigurd Ulveseth"|
|104| <Pole_Position_(video_game)>| "Pole Position"|
|10| <Bang_(band)>| "Bang"|
|44| <George_Susce_(catcher)>| "George Susce"|
|11| \<Kyle_Rankin>| "Kyle Rankin"|
|…|…|…|

Donc, la premier colonne correspond au nombre de fois que la chaîne de caractères de la troisième colonne a été utilisé pour faire allusion à la page Wikipédia de la deuxième colonne. C’est-à-dire que "Best Supporting Actor" a été utilisé 162 fois dans Wikipédia pour faire allusion à la page Wikipédia https://en.wikipedia.org/wiki/Saturn_Award_for_Best_Supporting_Actor . Le fichier anchor_text_en.tll  contient les informations de la Wikipédia en anglais et pas de la Wikipédia simple, donc certain titres n’ont pas de correspondance.

In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.8 MB/s eta 0:00:00


In [59]:
import pyterrier as pt
import Levenshtein

if not pt.started():
    pt.init()

# Définir une liste des termes de l'index
index_terms = ["Thomas", "Mario", "strikers", "playing", "Munich"]  # Remplacez ces exemples par les termes réels de votre index.

# Liste des mots à exclure
stop_words = ["and", "in"]  # Ajoutez les mots que vous voulez exclure ici.

# Fonction pour trouver le mot le plus proche dans l'index en termes de distance de Levenshtein
def correct_term(term, index_terms):
    # Exclure les mots de liaison de la correction
    if term.lower() in stop_words:
        return term

    closest_term = min(index_terms, key=lambda x: Levenshtein.distance(term, x))
    distance = Levenshtein.distance(term, closest_term)

    # Seulement retourner le terme corrigé si la distance est en dessous d'un certain seuil, par exemple 2.
    return closest_term if distance < 2 else term

def systeme(paragraph, model, index):
    res = {}
    mots = paragraph.split()

    br = pt.BatchRetrieve(index, wmodel=model, metadata=["docno", "title", "url"])

    for mot in mots:
        # Corriger le mot en utilisant la distance de Levenshtein par rapport à votre index_terms
        mot_correct = correct_term(mot, index_terms)

        # Effectuer la recherche avec le terme corrigé et ajouter les résultats
        res[mot] = br.search(mot_correct).head(10)

    # Afficher les résultats
    for token, results in res.items():
        print(f"Results for '{token}':")
        for idx, row in results.iterrows():
            print(f"{row['docno']} - {row['title']} (URL: {row['url']}")

    return res


# Exemple d'utilisation de la fonction
text = "Cristano Ronaldo is the best player"
results = systeme(text, "BM25", indexref2)

Results for 'Cristano':
Results for 'Ronaldo':
65655 - Cristiano Ronaldo (URL: https://simple.wikipedia.org/wiki/Cristiano%20Ronaldo
772767 - Cristiano Ronaldo International Airport (URL: https://simple.wikipedia.org/wiki/Cristiano%20Ronaldo%20International%20Airport
832515 - C.D. Nacional (URL: https://simple.wikipedia.org/wiki/C.D.%20Nacional
845434 - 2009 FIFA Pusks Award (URL: https://simple.wikipedia.org/wiki/2009%20FIFA%20Pusk%C3%A1s%20Award
863231 - Ronaldo Zamora (URL: https://simple.wikipedia.org/wiki/Ronaldo%20Zamora
793196 - Seven Sundays (2017 movie) (URL: https://simple.wikipedia.org/wiki/Seven%20Sundays%20%282017%20movie%29
867449 - 2007-08 UEFA Champions League (URL: https://simple.wikipedia.org/wiki/2007-08%20UEFA%20Champions%20League
839060 - Tallaght Stadium (URL: https://simple.wikipedia.org/wiki/Tallaght%20Stadium
454558 - Eden Hazard (URL: https://simple.wikipedia.org/wiki/Eden%20Hazard
787508 - Mustafa Azadzoy (URL: https://simple.wikipedia.org/wiki/Mustafa%20Azad

In [60]:
'''
Explication :
Ce code implémente un système de correction orthographique et de recherche d'informations utilisant la bibliothèque PyTerrier pour l'indexation et la recherche.
Il utilise la distance de Levenshtein pour corriger les mots mal orthographiés dans un paragraphe donné, puis effectue une recherche dans l'index pour récupérer
 les documents pertinents associés aux mots corrigés. Les résultats de la recherche sont affichés pour chaque mot du paragraphe.
 L'exemple d'utilisation montre comment le système peut être appelé avec un texte donné et un modèle de recherche spécifique.

'''

"\nExplication :\nCe code implémente un système de correction orthographique et de recherche d'informations utilisant la bibliothèque PyTerrier pour l'indexation et la recherche. \nIl utilise la distance de Levenshtein pour corriger les mots mal orthographiés dans un paragraphe donné, puis effectue une recherche dans l'index pour récupérer\n les documents pertinents associés aux mots corrigés. Les résultats de la recherche sont affichés pour chaque mot du paragraphe. \n L'exemple d'utilisation montre comment le système peut être appelé avec un texte donné et un modèle de recherche spécifique.\n\n"